In [1]:
import gym
import numpy
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
env = gym.make('Taxi-v2')
new_state = env.reset()
from copy import copy

[2017-12-04 12:50:48,550] Making new env: Taxi-v2


In [135]:
observation = env.reset()
for i in range(100):
    #env.render()
    #print (observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print ("Done")
        break

In [460]:
q_values = numpy.zeros((env.observation_space.n, env.action_space.n))
s_values = numpy.random.rand(env.observation_space.n)
gamma = 0.9
alpha = 0.5

In [346]:
def q_learn(iterations, q_values):
    total_reward = 0
    for i in range(iterations):
        new_state = env.reset()
        state = new_state 
        while True:
            action = numpy.argmax(q_values[state])
            new_state, reward, done, info = env.step(action)
            q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
            state = new_state
            total_reward += reward
            if done:
                break
    return total_reward

In [461]:
def dls(env_copy_1, new_state, curr_depth, max_depth):
    value_list = list()
    env_copy_2 = copy(env_copy_1)
    if curr_depth == max_depth:
        return s_values[new_state]
    else:
        for i in range(env.action_space.n):
            new_state_1, reward, done, info = env_copy_2.step(i)
                
            if done:
                value = s_values[new_state_1]
            else:
                value = reward + s_values[new_state] + gamma * dls(env_copy_2, new_state_1, curr_depth+1, max_depth)
            value_list.append(value)
            env_copy_2 = copy(env_copy_1)
        value = numpy.mean(value_list)
    return value

    

def dls_action(env_copy, state, max_depth):
    # Do depth Limited Search till max_depth on the mb_internal_environment
    value_list = numpy.zeros(env.action_space.n)
    for i in range(env.action_space.n):
        env_copy_1 = copy(env_copy)
        new_state = state
        value = 0
        for j in range(max_depth):
            new_state, reward, done, info = env_copy.step(numpy.argmax(q_values[new_state]))
            value += gamma**(1/(j+1))*(reward + numpy.argmax(q_values[new_state]))
        value_list[i] = value
    action = numpy.random.choice(numpy.flatnonzero(value_list == value_list.max()))
    return action

In [528]:
def dual(env, observation, max_depth, num_iters, condition):
    j = 0
    sum_reward = 0
    count = 0
    reward_list = list()
    for i in range(num_iters):
        new_state = env.reset()  
        env_copy = copy(env)        
        state = new_state
        sum_reward = 0
        while True:
            j += 1
            if condition:
                value_list = numpy.zeros(env.action_space.n)
                for k in range(env.action_space.n):
                    value = 0
                    next_state = state
                    for m in range(max_depth):
                        if m == 0:
                            next_state, search_reward, done, info = env_copy.step(k)
                        else:
                            next_state, search_reward, done, info = env_copy.step(numpy.argmax(q_values[next_state]))
                        value += gamma**(1/(m+1))*(search_reward + numpy.max(q_values[next_state]))
                    value_list[k] = value
                action = numpy.random.choice(numpy.flatnonzero(value_list == value_list.max()))
                new_state, reward, done, info = env.step(action)
                env_copy = copy(env)
                q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
                sum_reward += reward
                state = new_state
                count += 1
            else:
                action = numpy.argmax(q_values[state])
                new_state, reward, done, info = env.step(action)
                q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
                state = new_state
                sum_reward += reward
                count += 1
            if done:
                print(sum_reward)
                break
        reward_list.append(sum_reward)
    return reward_list, count
                

In [529]:
action = env.action_space.sample()
observation = env.reset()


In [543]:
condition = True
condition = False


In [548]:

values = dual(env, observation, max_depth=5, num_iters=500, condition=condition)

5
6
7
11
6
9
7
6
2
3
10
6
10
8
14
8
11
9
10
6
11
10
6
5
10
13
9
13
12
12
9
7
6
6
11
2
8
5
4
11
6
8
12
4
12
11
7
8
6
11
3
13
9
11
13
8
12
5
9
9
9
9
3
9
4
7
8
8
9
10
8
-281
3
5
12
4
9
8
2
9
8
7
8
3
6
3
4
7
-200
9
8
11
4
10
10
3
4
13
11
11
10
4
6
8
7
9
7
13
8
8
7
5
8
1
8
9
9
8
-290
6
5
8
7
14
6
10
9
11
13
6
8
14
10
6
9
8
9
3
10
13
6
8
9
10
11
12
3
3
6
11
6
11
3
9
6
6
12
5
13
8
4
4
9
7
11
6
14
5
6
6
10
7
9
4
6
6
13
9
12
7
8
8
6
4
6
5
10
12
8
6
9
14
7
6
6
7
10
10
9
12
9
11
14
2
3
3
7
1
8
8
10
6
8
9
4
4
7
9
-200
11
7
5
8
8
5
6
5
5
8
10
7
1
10
7
11
11
2
5
7
7
4
10
6
7
9
11
9
9
5
10
7
5
-200
13
7
8
9
3
8
8
9
4
5
6
4
7
9
4
9
7
5
7
10
5
10
5
10
6
9
4
7
10
14
10
6
6
9
9
6
7
8
10
7
10
8
7
3
7
9
5
6
4
9
10
13
4
12
6
12
5
5
4
9
12
12
6
7
8
10
5
7
7
7
8
9
4
12
1
14
5
-200
7
6
9
13
9
12
4
10
8
6
10
5
6
4
4
8
9
7
7
11
4
7
12
11
8
3
11
-200
11
11
14
9
12
4
12
5
9
13
4
3
5
3
6
3
9
7
7
9
11
4
8
10
10
8
6
4
3
9
12
11
9
10
8
10
6
14
11
10
9
12
-200
5
12
8
8
8
6
10
6
9
5
11
8
10
8
13
5
6
3
12
5
8
7
11
5
8
14

In [549]:
print (numpy.mean(values[0]), values[1])

3.64 8320


In [483]:
print (numpy.nonzero(q_values))
print (q_values[416])

(array([  1,   1,   1, ..., 499, 499, 499]), array([0, 1, 2, ..., 3, 4, 5]))
[ -6.5         -6.01293945  -6.25        -6.5        -10.          -6.15234375]


In [ ]:
env.reset()
print (q_learn(1, q_values))

In [ ]:
print (s_values)